In [2]:
!pip install transformers[torch] -q

In [3]:
!pip install datasets -q --upgrade

In [4]:
!pip install evaluate -q

In [5]:
!pip install wandb -q --upgrade

In [6]:
import os
import gc
import math
import librosa
import numpy as np
import pandas as pd
import librosa as lb
import matplotlib.pyplot as plt

from scipy import signal
from scipy.fft import fftshift

from collections import defaultdict
from itertools import islice
from typing import Any
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, precision_score, recall_score

import torch
import torchaudio

from IPython import display
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
from tqdm.auto import trange
from tqdm import tqdm
tqdm.pandas()

import re
import seaborn as sns
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import (
    load_dataset,
    load_metric,
    load_from_disk,
    Audio,
    concatenate_datasets
)
from transformers import (
    AutoConfig,
    Wav2Vec2Processor,
    Wav2Vec2Tokenizer,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2Model,
    TrainingArguments,
    Trainer,
    get_linear_schedule_with_warmup,
    EvalPrediction
)

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)
import evaluate

2024-05-27 09:31:32.963182: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 09:31:32.963289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 09:31:33.086182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
torch.__version__

'2.1.2'

In [11]:
import transformers
transformers.__version__

'4.39.3'

In [7]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
import os
os.environ["WANDB_PROJECT"]="thesis_model_batch"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [9]:
import datasets
datasets.__version__

'2.19.1'

In [10]:
train_path = '/kaggle/input/thesis-train-data/train'
train1_path = '/kaggle/input/thesis-model-batch-3sec-8/train1'
train2_path = '/kaggle/input/thesis-model-batch-3sec-8/train1'

val_path = '/kaggle/input/thesis-model-batch-3sec-8/val_2sec_8.pickle'
path_to_save = '/kaggle/working/second_model.pth'

model_name = 'facebook/wav2vec2-large-xlsr-53'
run_name = 'basic_model_one_tahn_russian'

In [ ]:
train1 = load_from_disk(train1_path)
train2 = load_from_disk(train2_path)

val_pd = pd.read_pickle(val_path)

In [12]:
from datasets import concatenate_datasets
train = concatenate_datasets([train1, train2])

In [15]:
class CustomDataset(Dataset):

    def __init__(self, input_values, labels, audio_paths):
        super().__init__()
        self.input_values = input_values
        self.labels = labels
        self.audio_paths = audio_paths

    def __len__(self):
        return len(self.input_values)

    def __getitem__(self, idx):
        return {'input_values': self.input_values[idx].tolist(), 'labels': self.labels[idx], 'audio_ID': self.audio_paths[idx]}

In [16]:
val = CustomDataset(
    input_values = val_pd['input_values'].tolist(),
    labels = val_pd['labels'].tolist(),
    audio_paths = val_pd['audio_ID'].tolist()
)

In [17]:
import gc
del val_pd
gc.collect()

1364

## Initialize model

In [23]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [39]:
# https://github.com/m3hrdadfi/soxan/blob/main/notebooks/Emotion_recognition_in_Greek_speech_using_Wav2Vec2.ipynb
import torch
from torch import nn
from torch.nn import functional as nnf
from tqdm import tqdm
tqdm.pandas()

from transformers import (
    Wav2Vec2Model
)
from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(1024, 256)
        self.mid_proj = nn.Linear(256, config['hidden_size'])
        self.hidden_dropout = nn.Dropout(config['hidden_dropout'])
        self.final_dropout = nn.Dropout(config['final_dropout'])
        self.out_proj = nn.Linear(config['hidden_size'], config['num_labels'])
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def forward(self, features, **kwargs):
        x = self.hidden_dropout(features)
        x = self.dense(x)
        x = self.relu(x)
        x = self.mid_proj(x)
        x = self.relu(x)
        x = self.final_dropout(x)
        x = self.out_proj(x)
        return x

class Wav2VecClassifer(Wav2Vec2PreTrainedModel):

    def __init__(self, w2v_config, config):
        super().__init__(w2v_config)

        self.num_labels = config['num_labels']
        self.pooling_strategy = config['pooling_mode']
        self.wav2vec2 = Wav2Vec2Model.from_pretrained(
            config['model_name'], 
            config=w2v_config
        )
        self.proj_layer = nn.Linear(config['input_size'], 256)
        self.classifier = Wav2Vec2ClassificationHead(config)
        self.loss = nn.CrossEntropyLoss()
        self.post_init()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            labels,
            attention_mask=None,
            device=None,
            output_attentions=False,
            output_hidden_states=False,
            number_of_chunks=None
    ):

        number_of_chunks = [len(input_values)]

        outputs = self.wav2vec2(
            input_values=input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states
        )

        hidden_states = self.merged_strategy(outputs.last_hidden_state, mode='mean')
        # # var 1: pooled_over model
        logits = self.classifier(hidden_states)
        preds = logits.flatten().view(-1, 2)
        preds_split = preds.split(number_of_chunks)

        if self.pooling_strategy == "mean":
            pooled_preds = torch.cat([torch.mean(x, dim=0) for x in preds_split])
        elif self.pooling_strategy == "max":
            pooled_preds = torch.cat([torch.max(x).reshape(1).repeat(len(x)) for x in preds_split])

        #var 2: batched model
        hidden_states = outputs.last_hidden_state
        all_hs = []
        for hs in hidden_states:
            all_hs.append(hs.view(1, -1, 1))
        chunks_num = len(all_hs) - 1

        concat_states = torch.cat(all_hs, dim=2)
        raw = nnf.fold(concat_states, (249+100*chunks_num, 1024), kernel_size=(249, 1024), stride=(100, 1024))
        counter = nnf.fold(torch.ones_like(concat_states), (249+100*chunks_num, 1024), kernel_size=(249, 1024), stride=(100, 1024))
        result = raw / counter
        avg_state = result.squeeze(0).mean(dim=1)
        pooled_preds = self.classifier(avg_state)

        loss = None
        if labels is not None:
            loss = self.loss(pooled_preds.view(-1, 2), labels.view(-1))
        output = (pooled_preds,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [25]:
config = {
    'output_hidden_states': False,
    'num_labels': 2,
    'hidden_size': 64,
    'final_dropout': 0.1,
    'hidden_dropout': 0.2,
    'pooling_mode': 'mean',
    'model_name': 'jonatasgrosman/wav2vec2-large-xlsr-53-russian',
    'input_size': 1024
}

model_args_w2v2 = {
    "attention_dropout": 0.1,
    "hidden_dropout": 0.1,
    "feat_proj_dropout": 0.0,
    "mask_time_prob": 0.05,
    "layerdrop": 0.1,
    'layer_norm_eps':1e-4,
}

model_args_w2v2_config = Wav2Vec2Config.from_pretrained(
    config['model_name'], 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
#     layer_norm_eps:1e-4,
)

model = Wav2VecClassifer(model_args_w2v2_config, config).cuda()

config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model.freeze_feature_extractor()

### Trainer

In [28]:
from torch import Tensor

def collate_fn_pooled_tokens(data):
    input_ids = [torch.tensor(data[i]['input_values']) for i in range(len(data))]
    attention_mask = [torch.where(input_ids[i]==0, 0, 1) for i in range(len(input_ids))]
    label_features = [data[i]['labels'] for i in range(len(data))]
    d_type = torch.long if isinstance(label_features[0], int) else torch.float
    if len(data[0]) == 2:
        collated = {
            'input_values': input_ids[0],
            'attention_mask': attention_mask[0],
            'labels': torch.tensor(label_features, dtype=d_type)
        }
    else:
        collated = {
            'input_values': input_ids[0],
            'attention_mask': attention_mask[0],
            'labels': torch.tensor(label_features, dtype=d_type)
        }
    return collated

In [29]:
processor = Wav2Vec2FeatureExtractor.from_pretrained('facebook/wav2vec2-large-xlsr-53')
data_collator = collate_fn_pooled_tokens

preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [31]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    labels = p.label_ids

    cr = evaluate.load('bstrai/classification_report')
    result = cr.compute(predictions=preds, references=labels, labels=[0, 1])
    
    res_dict = {}
    for label in result.keys():
        if isinstance(result[label], dict):
            for metric in result[label].keys():
                res_dict[f'{metric}_{label}'] = result[label][metric]
        else:
            res_dict[label] = result[label]

    return res_dict

In [32]:
import random
def set_seed(seed: int, deterministic: bool = False):
        """
        Helper function for reproducible behavior to set the seed in `random`, `numpy`, `torch` and/or `tf` (if installed).

        Args:
            seed (`int`):
                The seed to set.
            deterministic (`bool`, *optional*, defaults to `False`):
                Whether to use deterministic algorithms where available. Can slow down training.
        """
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
        if deterministic:
            torch.use_deterministic_algorithms(True)

set_seed(888)

In [33]:
from transformers import TrainingArguments

In [34]:
num_epochs = 2
warmup_ratio = 0.1
total_steps = len(train) * num_epochs

In [35]:
len(val)

1128

In [ ]:
# for lr in [1e-3, 2e-3, 1e-4]
lr = 1e-3
config = {
    'output_hidden_states': False,
    'num_labels': 2,
    'hidden_size': 64,
    'final_dropout': 0.1,
    'hidden_dropout': 0.2,
    'pooling_mode': 'mean',
    'model_name': 'jonatasgrosman/wav2vec2-large-xlsr-53-russian',
    'input_size': 1024
}

model_args_w2v2 = {
    "attention_dropout": 0.1,
    "hidden_dropout": 0.1,
    "feat_proj_dropout": 0.0,
    "mask_time_prob": 0.05,
    "layerdrop": 0.1,
    'layer_norm_eps':1e-4,
}

model_args_w2v2_config = Wav2Vec2Config.from_pretrained(
    config['model_name'], 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
#     layer_norm_eps:1e-4,
)

model = Wav2VecClassifer(model_args_w2v2_config, config).cuda()
model.freeze_feature_extractor()

optimizer = torch.optim.AdamW([
        {"params": model.wav2vec2.parameters(), "lr": 3e-4},
    ], lr=lr)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps * warmup_ratio,
    num_training_steps=total_steps
)

training_args = TrainingArguments(
    output_dir=f"/content/model_one_trainer_{lr}",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="steps",
    num_train_epochs=num_epochs,
    fp16=True,
    save_steps=5000,
    eval_steps=2500,
    learning_rate=lr,
    save_total_limit=3,
    report_to='wandb',
    run_name=f"model_2_{lr}_w2v_3e-4_pooled_over_more_layers",
    logging_steps=1000,
    load_best_model_at_end=True,
    seed=888,
    save_only_model=True
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=processor,
    optimizers=(optimizer, scheduler)
)

trainer.train()

In [41]:
wandb.finish()

eval/accuracy,▁█▇▇█▂▄
eval/f1-score_0,█████▁▆
eval/f1-score_1,▁▇▇▆▇█▇
eval/f1-score_macro avg,▁██▇█▂▆
eval/f1-score_weighted avg,▁██▇█▂▆
eval/loss,▆▄▃▂▁█▄
eval/precision_0,▅██▇█▁▇
eval/precision_1,▁▇▇██▆▆
eval/precision_macro avg,▁████▄▆
eval/precision_weighted avg,▁████▄▆
eval/recall_0,█▆▆▇▇▁▄


In [ ]:
assert False

In [ ]:
# for lr in [1e-3, 2e-3, 1e-4]
lr = 2e-3
config = {
    'output_hidden_states': False,
    'num_labels': 2,
    'hidden_size': 64,
    'final_dropout': 0.1,
    'hidden_dropout': 0.2,
    'pooling_mode': 'mean',
    'model_name': 'jonatasgrosman/wav2vec2-large-xlsr-53-russian',
    'input_size': 1024
}

model_args_w2v2 = {
    "attention_dropout": 0.1,
    "hidden_dropout": 0.1,
    "feat_proj_dropout": 0.0,
    "mask_time_prob": 0.05,
    "layerdrop": 0.1,
    'layer_norm_eps':1e-4,
}

model_args_w2v2_config = Wav2Vec2Config.from_pretrained(
    config['model_name'], 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
#     layer_norm_eps:1e-4,
)

model = Wav2VecClassifer(model_args_w2v2_config, config).cuda()
model.freeze_feature_extractor()

optimizer = torch.optim.AdamW([
        {"params": model.wav2vec2.parameters(), "lr": 1e-5},
#         {"params": model.classification_head.parameters(), "lr": 1e-3}
    ], lr=lr)

scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=total_steps * warmup_ratio,
            num_training_steps=total_steps
)

training_args = TrainingArguments(
    output_dir=f"/content/model_one_trainer_{lr}",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
#     gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=2,
    fp16=True,
    save_steps=5000,
    eval_steps=500,
    learning_rate=lr,
    save_total_limit=3,
    report_to='wandb',
    run_name=f"model_2_{lr}_7500",
    logging_steps=1000,
    warmup_steps=7500,
    warmup_ratio=0.1,
    load_best_model_at_end=True,
#     label_smoothing_factor=1e-5,
    seed=888
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=processor,
    optimizers=(optimizer, scheduler)
)

trainer.train()

In [ ]:
wandb.finish()

In [ ]:
# for lr in [1e-3, 2e-3, 1e-4]
lr = 1e-4
config = {
    'output_hidden_states': False,
    'num_labels': 2,
    'hidden_size': 64,
    'final_dropout': 0.1,
    'hidden_dropout': 0.2,
    'pooling_mode': 'mean',
    'model_name': 'jonatasgrosman/wav2vec2-large-xlsr-53-russian',
    'input_size': 1024
}

model_args_w2v2 = {
    "attention_dropout": 0.1,
    "hidden_dropout": 0.1,
    "feat_proj_dropout": 0.0,
    "mask_time_prob": 0.05,
    "layerdrop": 0.1,
    'layer_norm_eps':1e-4,
}

model_args_w2v2_config = Wav2Vec2Config.from_pretrained(
    config['model_name'], 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
#     layer_norm_eps:1e-4,
)

model = Wav2VecClassifer(model_args_w2v2_config, config).cuda()
model.freeze_feature_extractor()

optimizer = torch.optim.AdamW([
        {"params": model.wav2vec2.parameters(), "lr": 1e-5},
    ], lr=lr)

scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=total_steps * warmup_ratio,
            num_training_steps=total_steps
)

training_args = TrainingArguments(
    output_dir=f"/content/model_one_trainer_{lr}",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
#     gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=2,
    fp16=True,
    save_steps=1000,
    eval_steps=500,
    learning_rate=lr,
    save_total_limit=3,
    report_to='wandb',
    run_name=f"model_2_{lr}_7500",
    logging_steps=1000,
    warmup_steps=7500,
    warmup_ratio=0.1,
    load_best_model_at_end=True,
#     label_smoothing_factor=1e-5,
    seed=888
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=val,
    tokenizer=processor,
    optimizers=(optimizer, None)
)

trainer.train()

In [ ]:
assert False